# Hitmaps 2024-06-06 - 234m Station

#### Import Libraries

In [ ]:
import ROOT
import pps_hitmaps
from array import array
from pathlib import Path
import pandas
import numpy
from math import ceil
import Helper20240606

## Get Sensor

In [ ]:
new_sensor = pps_hitmaps.RectangularPadSensor(NumSmallerCols = 16)
new_sensor_ti = pps_hitmaps.RectangularPadSensor(NumSmallerCols = 16, PadSpacing=0.01)
fig = new_sensor.preview()

## Load Hitmaps and check everything is ok

In [ ]:
from pathlib import Path

backgroundFlux = 5*10**12
backgroundTitle = "5E12 p/cm^2"
#noBackgroundFlux = 200  # This is needed because when making the sensor loss probability maps, we were running into numerican stability issues, giving weird results. Adding this small numbers, makes the behaviour consistent
noBackgroundFlux = 0

base_dir = Path("./2024-06-06-Hitmaps")
vertical_dir   = base_dir/"maps-vertical"
horizontal_dir = base_dir/"maps-horizontal"

station = "234"
angle_beta = [
    ("horizontal", 125, 0.15),  # Angle dir, angle in urad, betastar in m
    ("horizontal", 125, 0.50),
    ("horizontal", 250, 0.15),
    ("horizontal", 250, 0.50),
    ("vertical", 125, 0.20),
    ("vertical", 125, 0.50),
    ("vertical", 250, 0.20),
    ("vertical", 250, 0.50),
]

hitmaps = {}

for angle_dir, angle, beta in angle_beta:
    dir_key = "v"
    if angle_dir == "horizontal":
        dir_key = "h"

    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    if dir_key == "v":
        directory = vertical_dir
    else:
        directory = horizontal_dir

    # Find the hitmap file
    hitmap_files = []
    for file in directory.glob(f"map-{dir_key}{angle}urad-{int(beta*100)}cm-physics-sd+el-{station}-nodetcut_2024*"):
        hitmap_files += [file]
    if len(hitmap_files) == 0:
        print(f"Could not find a hitmap file for key {hitmap_key}")
        continue
    if len(hitmap_files) > 1:
        print(f"Found more than one hitmap file for key {hitmap_key}")
        continue

    hitmap_file = hitmap_files[0]

    hitmaps[hitmap_key] = pps_hitmaps.PPSHitmap(
        hitmap_file,
        station,
        Helper20240606.detector_edge[station][beta],
        betastar = beta,
        yStep=0.000025,
        xStep=0.000025,
        verbose = False,
        addBackgroundFlux = noBackgroundFlux,
    )
    hitmaps[hitmap_key+"-background"] = pps_hitmaps.PPSHitmap(
        hitmap_file,
        station,
        Helper20240606.detector_edge[station][beta],
        betastar = beta,
        yStep=0.000025,
        xStep=0.000025,
        verbose = False,
        addBackgroundFlux = backgroundFlux,
    )


#test_hitmap = hitmaps["196-physics"]

print("Validating the hitmap files:")
for map in hitmaps:
    print("  - Map {}".format(map))
    hitmaps[map].validate()

## Fluxmaps

### Create Histograms

In [ ]:
hitmap_histograms = {}

for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    hitmap_histograms[hitmap_key] = {
        "physics": hitmaps[hitmap_key].getHisto(
            f"station{station}-{angle_dir}-{angle}urad-{int(beta*100)}cm",
            f"Station {station} - {angle_dir} crossing angle, {angle} urad; {int(beta*100)}cm Beta star",
                                                ),
        "background": hitmaps[hitmap_key+"-background"].getHisto(
            f"station{station}-background-{angle_dir}-{angle}urad-{int(beta*100)}cm",
            f"Station {station} with {backgroundTitle} Background - {angle_dir} crossing angle, {angle} urad; {int(beta*100)}cm Beta star",
                                                ),
    }


### Plot Histograms

In [ ]:
def drawFluxmapWithBackground(angle_dir, angle):
    canv = ROOT.TCanvas(f"{angle_dir}_{angle}urad_detectorEdge", f"{angle_dir}_{angle}urad_detectorEdge", 1200, 600)
    canv.Divide(2,1)

    betas = [0.50, 0.20]
    if angle_dir == "horizontal":
        betas = [0.50, 0.15]

    idx = 0
    memory = {}
    for beta in betas:
        hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
        idx += 1
        pad = canv.cd(idx)

        pad.SetTicks()
        pad.SetLogz()
        pad.SetLeftMargin(0.11)
        pad.SetRightMargin(0.16)
        pad.SetTopMargin(0.07)
        pad.SetBottomMargin(0.14)

        memory[hitmap_key] = {}
        memory[hitmap_key]["histogram"] = hitmap_histograms[hitmap_key]["physics"].Clone()
        memory[hitmap_key]["histogram"].SetTitle(f"$\\beta^* = {int(beta*100)} \, \\text{{cm}}$")

        memory[hitmap_key]["histogram"].Draw("colz")

        edge = hitmaps[hitmap_key].detectorEdge * 1000 # Convert to mm for drawing
        yMin = hitmaps[hitmap_key].yMin * 1000         # Convert to mm for drawing
        yMax = hitmaps[hitmap_key].yMax * 1000         # Convert to mm for drawing
        memory[hitmap_key]["line"] = ROOT.TLine(edge,yMin,edge,yMax)
        memory[hitmap_key]["line"].SetLineColor(ROOT.kRed)
        memory[hitmap_key]["line"].Draw("same")
    return canv, memory

#### Vertical 250 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("vertical", 250)

canv.Draw()

#### Vertical 125 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("vertical", 125)

canv.Draw()

#### Horizontal 250 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("horizontal", 250)

canv.Draw()

#### Horizontal 125 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("horizontal", 125)

canv.Draw()

### Plot Histograms with Background

In [ ]:
def drawFluxmapWithBackground(angle_dir, angle):
    canv = ROOT.TCanvas(f"{angle_dir}_{angle}urad_background", f"{angle_dir}_{angle}urad_background", 1200, 1200)
    canv.Divide(2,2)

    betas = [0.50, 0.20]
    if angle_dir == "horizontal":
        betas = [0.50, 0.15]

    idx = 0
    memory = {}
    for beta in betas:
        hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
        idx += 1

        memory[hitmap_key] = {}
        memory[hitmap_key]["histogram"] = hitmap_histograms[hitmap_key]["physics"].Clone()
        memory[hitmap_key]["histogram"].SetTitle(f"$\\beta^* = {int(beta*100)} \, \\text{{cm}}$")
        memory[hitmap_key]["background-histogram"] = hitmap_histograms[hitmap_key]["background"].Clone()
        memory[hitmap_key]["background-histogram"].SetTitle(f"$\\beta^* = {int(beta*100)} \, \\text{{cm}}$")
        memory[hitmap_key]["background-histogram"].SetMinimum(backgroundFlux*0.9)

        edge = hitmaps[hitmap_key].detectorEdge * 1000 # Convert to mm for drawing
        yMin = hitmaps[hitmap_key].yMin * 1000         # Convert to mm for drawing
        yMax = hitmaps[hitmap_key].yMax * 1000         # Convert to mm for drawing
        memory[hitmap_key]["line"] = ROOT.TLine(edge,yMin,edge,yMax)
        memory[hitmap_key]["line"].SetLineColor(ROOT.kRed)

        pad = canv.cd((idx-1)*2+1)
        pad.SetTicks()
        pad.SetLogz()
        pad.SetLeftMargin(0.11)
        pad.SetRightMargin(0.16)
        pad.SetTopMargin(0.07)
        pad.SetBottomMargin(0.14)
        memory[hitmap_key]["histogram"].Draw("colz")
        memory[hitmap_key]["line"].Draw("same")

        pad = canv.cd((idx-1)*2+2)
        pad.SetTicks()
        pad.SetLogz()
        pad.SetLeftMargin(0.11)
        pad.SetRightMargin(0.16)
        pad.SetTopMargin(0.07)
        pad.SetBottomMargin(0.14)
        memory[hitmap_key]["background-histogram"].Draw("colz")
        memory[hitmap_key]["line"].Draw("same")
    return canv, memory

#### Vertical 250 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("vertical", 250)

canv.Draw()

#### Vertical 125 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("vertical", 125)

canv.Draw()

#### Horizontal 250 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("horizontal", 250)

canv.Draw()

#### Horizontal 125 urad

In [ ]:
canv, persistance = drawFluxmapWithBackground("horizontal", 125)

canv.Draw()

### Overlaid Sensor Nominal Positions

In [ ]:
sensor = new_sensor_ti
xSensorSize = sensor.maxX - sensor.minX
ySensorSize = sensor.maxY - sensor.minY

nominal_positions = Helper20240606.getNominalPositions(hitmaps, xSensorSize)

In [ ]:
def drawFluxmapWithSensor(angle_dir, angle, sensor_positions = None):
    canv = ROOT.TCanvas(f"{angle_dir}_{angle}urad_overlaidSensor", f"{angle_dir}_{angle}urad_overlaidSensor", 1200, 600)
    canv.Divide(2,1)

    betas = [0.50, 0.20]
    if angle_dir == "horizontal":
        betas = [0.50, 0.15]

    idx = 0
    memory = {}
    for beta in betas:
        hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
        position_key = f"{angle_dir}-{angle}urad"
        idx += 1
        pad = canv.cd(idx)

        pad.SetTicks()
        pad.SetLogz()
        pad.SetLeftMargin(0.11)
        pad.SetRightMargin(0.16)
        pad.SetTopMargin(0.07)
        pad.SetBottomMargin(0.14)

        if sensor_positions is not None:
            offsetX, offsetY = sensor_positions[position_key]
        else:
            offsetY = hitmaps[hitmap_key].maxFluence["y"]*1000
        offsetX = hitmaps[hitmap_key].detectorEdge*1000 + xSensorSize/2

        memory[hitmap_key] = {}
        memory[hitmap_key]["histogram"] = hitmap_histograms[hitmap_key]["physics"].Clone()
        memory[hitmap_key]["histogram"].SetTitle(f"$\\beta^* = {int(beta*100)} \, \\text{{cm}}$")

        memory[hitmap_key]["histogram"].Draw("colz")

        edge = hitmaps[hitmap_key].detectorEdge * 1000 # Convert to mm for drawing
        yMin = hitmaps[hitmap_key].yMin * 1000         # Convert to mm for drawing
        yMax = hitmaps[hitmap_key].yMax * 1000         # Convert to mm for drawing
        memory[hitmap_key]["line"] = ROOT.TLine(edge,yMin,edge,yMax)
        memory[hitmap_key]["line"].SetLineColor(ROOT.kRed)
        memory[hitmap_key]["line"].Draw("same")

        memory[hitmap_key]["sensorLines"] = {}
        memory[hitmap_key]["sensorLines"]["Left"]   = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                                  sensor.minX + offsetX, sensor.maxY + offsetY)
        memory[hitmap_key]["sensorLines"]["Right"]  = ROOT.TLine( sensor.maxX + offsetX, sensor.minY + offsetY,
                                                                  sensor.maxX + offsetX, sensor.maxY + offsetY)
        memory[hitmap_key]["sensorLines"]["Top"]    = ROOT.TLine( sensor.minX + offsetX, sensor.minY + offsetY,
                                                                  sensor.maxX + offsetX, sensor.minY + offsetY)
        memory[hitmap_key]["sensorLines"]["Bottom"] = ROOT.TLine( sensor.minX + offsetX, sensor.maxY + offsetY,
                                                                  sensor.maxX + offsetX, sensor.maxY + offsetY)
        for key in memory[hitmap_key]["sensorLines"]:
            memory[hitmap_key]["sensorLines"][key].SetLineColor(ROOT.kRed)

        padID = 0
        for pad in sensor.padVec:
            minX = pad.minX
            maxX = pad.maxX
            minY = pad.minY
            maxY = pad.maxY

            memory[hitmap_key]["sensorLines"][f"pad{padID}_Left"]   = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                                  minX + offsetX, maxY + offsetY)
            memory[hitmap_key]["sensorLines"][f"pad{padID}_Right"]  = ROOT.TLine( maxX + offsetX, minY + offsetY,
                                                                                  maxX + offsetX, maxY + offsetY)
            memory[hitmap_key]["sensorLines"][f"pad{padID}_Top"]    = ROOT.TLine( minX + offsetX, minY + offsetY,
                                                                                  maxX + offsetX, minY + offsetY)
            memory[hitmap_key]["sensorLines"][f"pad{padID}_Bottom"] = ROOT.TLine( minX + offsetX, maxY + offsetY,
                                                                                  maxX + offsetX, maxY + offsetY)

            memory[hitmap_key]["sensorLines"][f"pad{padID}_Left"].SetLineColor(ROOT.kBlue)
            memory[hitmap_key]["sensorLines"][f"pad{padID}_Right"].SetLineColor(ROOT.kBlue)
            memory[hitmap_key]["sensorLines"][f"pad{padID}_Top"].SetLineColor(ROOT.kBlue)
            memory[hitmap_key]["sensorLines"][f"pad{padID}_Bottom"].SetLineColor(ROOT.kBlue)

            padID += 1

        for key in memory[hitmap_key]["sensorLines"]:
            memory[hitmap_key]["sensorLines"][key].Draw("same")

    return canv, memory

#### Vertical 250 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("vertical", 250)

canv.Draw()

#### Vertical 125 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("vertical", 125)

canv.Draw()

#### Horizontal 250 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("horizontal", 250)

canv.Draw()

#### Horizontal 125 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("horizontal", 125)

canv.Draw()

#### Nominal Positions

In [ ]:
for key in nominal_positions:
    print(f"{key}: {nominal_positions[key]}")

### Overlaid Sensor Adjusted Positions

In [ ]:
sensor = new_sensor_ti
xSensorSize = sensor.maxX - sensor.minX
ySensorSize = sensor.maxY - sensor.minY

adjusted_positions = Helper20240606.getAdjustedPositions(nominal_positions, station)
for key in adjusted_positions:
    print(f"{key}: {adjusted_positions[key]}")

#### Vertical 250 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("vertical", 250, adjusted_positions)

canv.Draw()

#### Vertical 125 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("vertical", 125, adjusted_positions)

canv.Draw()

#### Horizontal 250 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("horizontal", 250, adjusted_positions)

canv.Draw()

#### Horizontal 125 urad

In [ ]:
canv, persistance = drawFluxmapWithSensor("horizontal", 125, adjusted_positions)

canv.Draw()

## Occupancy Plots

### Get Square Pad Occupancy and plot it

In [ ]:
square_occupancy_graphs = {}

for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    square_occupancy_graphs[hitmap_key] = {
        "physics": hitmaps[hitmap_key].squarePadOccupancy(),
        "background": hitmaps[hitmap_key+"-background"].squarePadOccupancy(),
    }

In [ ]:
for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    if hitmap_key not in square_occupancy_graphs:
        continue

    if "canv" not in square_occupancy_graphs[hitmap_key]:
        square_occupancy_graphs[hitmap_key]["canv"] = ROOT.TCanvas(f"square_occupancy_vs_padSize_{station}_{angle_dir}-{angle}urad-{int(beta*100)}cm", f"Station {station} - Occupancy vs Pad Size - {angle}urad {angle_dir} {int(beta*100)}cm", 1200, 600)
        square_occupancy_graphs[hitmap_key]["canv"].Divide(2,1)

        idx = 0
        for hitmap_type in ['physics', 'background']:
            idx += 1
            pad = square_occupancy_graphs[hitmap_key]["canv"].cd(idx)
            pad.SetLogx()
            pad.SetLogy()
            pad.SetTicks()

            title = f"{angle}urad {angle_dir} {int(beta*100)}cm - {station}m"
            if hitmap_type == 'background':
                title = f"{angle}urad {angle_dir} {int(beta*100)}cm - {station}m with {backgroundTitle} Background"

            if "frame" not in square_occupancy_graphs[hitmap_key][hitmap_type]:
                square_occupancy_graphs[hitmap_key][hitmap_type]["frame"] = ROOT.TH2D(f"frame_square_{station}_{angle_dir}-{angle}urad-{int(beta*100)}cm_{hitmap_type}", title, 100, 10, 10000, 100, 1.0E-5, 1)
                square_occupancy_graphs[hitmap_key][hitmap_type]["frame"].SetStats(False)
                square_occupancy_graphs[hitmap_key][hitmap_type]["frame"].GetXaxis().SetTitle("Pad Width [#mum]")
                square_occupancy_graphs[hitmap_key][hitmap_type]["frame"].GetYaxis().SetTitle("#mu")

            square_occupancy_graphs[hitmap_key][hitmap_type]["frame"].Draw()

            square_occupancy_graphs[hitmap_key][hitmap_type]["uniform"].Draw("l same")
            square_occupancy_graphs[hitmap_key][hitmap_type]["integrate"].Draw("l same")
            square_occupancy_graphs[hitmap_key][hitmap_type]["points"].SetMarkerStyle(20)
            square_occupancy_graphs[hitmap_key][hitmap_type]["points"].Draw("p same")

    square_occupancy_graphs[hitmap_key]["canv"].Draw()

### Get Rectangular Pad Occupancy and plot it

In [ ]:
rect_occupancy_graphs = {}

for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    rect_occupancy_graphs[hitmap_key] = {
        "physics": hitmaps[hitmap_key].rectangularPadOccupancy(xLen=1.3e-3),
        "background": hitmaps[hitmap_key+"-background"].rectangularPadOccupancy(xLen=1.3e-3),
    }

In [ ]:
for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    if hitmap_key not in rect_occupancy_graphs:
        continue

    if "canv" not in rect_occupancy_graphs[hitmap_key]:
        rect_occupancy_graphs[hitmap_key]["canv"] = ROOT.TCanvas(f"rect_occupancy_vs_padSize_{station}_{angle_dir}-{angle}urad-{int(beta*100)}cm", f"Station {station} - Occupancy vs Pad Size - {angle}urad {angle_dir} {int(beta*100)}cm", 1200, 600)
        rect_occupancy_graphs[hitmap_key]["canv"].Divide(2,1)

        idx = 0
        for hitmap_type in ['physics', 'background']:
            idx += 1
            pad = rect_occupancy_graphs[hitmap_key]["canv"].cd(idx)
            pad.SetLogx()
            pad.SetLogy()
            pad.SetTicks()

            title = f"{angle}urad {angle_dir} {int(beta*100)}cm - {station}m"
            if hitmap_type == 'background':
                title = f"{angle}urad {angle_dir} {int(beta*100)}cm - {station}m with {backgroundTitle} Background"

            if "frame" not in rect_occupancy_graphs[hitmap_key][hitmap_type]:
                rect_occupancy_graphs[hitmap_key][hitmap_type]["frame"] = ROOT.TH2D(f"frame_rect_{station}_{angle_dir}-{angle}urad-{int(beta*100)}cm_{hitmap_type}", title, 100, 10, 10000, 100, 1.0E-5, 1)
                rect_occupancy_graphs[hitmap_key][hitmap_type]["frame"].SetStats(False)
                rect_occupancy_graphs[hitmap_key][hitmap_type]["frame"].GetXaxis().SetTitle("Pad Width [#mum]")
                rect_occupancy_graphs[hitmap_key][hitmap_type]["frame"].GetYaxis().SetTitle("#mu")

            rect_occupancy_graphs[hitmap_key][hitmap_type]["frame"].Draw()

            rect_occupancy_graphs[hitmap_key][hitmap_type]["uniform"].Draw("l same")
            rect_occupancy_graphs[hitmap_key][hitmap_type]["integrate"].Draw("l same")
            rect_occupancy_graphs[hitmap_key][hitmap_type]["points"].SetMarkerStyle(20)
            rect_occupancy_graphs[hitmap_key][hitmap_type]["points"].Draw("p same")

    rect_occupancy_graphs[hitmap_key]["canv"].Draw()

In [ ]:
occupancy_graphs = {
    'square': square_occupancy_graphs,
    'rectangular': rect_occupancy_graphs,
}

## Event Loss Probabilities

### Convert Occupancy to Event Loss Probabilities

In [ ]:
event_loss_probabilities = {}

for pad_type in occupancy_graphs:
    if pad_type not in event_loss_probabilities:
        event_loss_probabilities[pad_type] = {}

    for hitmap_key in occupancy_graphs[pad_type]:
        if hitmap_key not in event_loss_probabilities[pad_type]:
            event_loss_probabilities[pad_type][hitmap_key] = {}

        for hitmap_type in occupancy_graphs[pad_type][hitmap_key]:
            if hitmap_type == "canv":
                continue
            event_loss_probabilities[pad_type][hitmap_key][hitmap_type] = pps_hitmaps.occupancyGraphToEventLossProbability(
                occupancy_graphs[pad_type][hitmap_key][hitmap_type]['integrate']
            )

### Plot Event Loss Probability vs Pad Dimensions

In [ ]:
prob_vs_dim_canvas = {}
prob_vs_dim_objs = {}

drawTimeSteps = [0, 1, 3, 7, 39, 399]

for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    do_continue = False
    for pad_type in occupancy_graphs:
        if hitmap_key not in occupancy_graphs[pad_type]:
            do_continue = True
            break
    if do_continue:
        continue

    if hitmap_key not in prob_vs_dim_objs:
        prob_vs_dim_objs[hitmap_key] = {}

    canv = ROOT.TCanvas(f"probability_vs_dimension_{hitmap_key}", f"{angle}urad {angle_dir} {int(beta*100)}cm Station {station} - Event Loss Probability vs Pad Dimensions", 1200, 1200)
    canv.Divide(2,2)

    idx = 0
    for pad_type in event_loss_probabilities:
        pad_title = "Square Pad"
        x_title = "Pad Size [#mum]"
        if pad_type == "rectangular":
            pad_title = "Rectangular Pad"
            x_title = "Pad Width [#mum]"

        if pad_type not in prob_vs_dim_objs[hitmap_key]:
            prob_vs_dim_objs[hitmap_key][pad_type] = {}

        for hitmap_type in event_loss_probabilities[pad_type][hitmap_key]:
            hitmap_title = ""
            if hitmap_type == 'background':
                hitmap_title = f" with {backgroundTitle} Background"

            if hitmap_type not in prob_vs_dim_objs[hitmap_key][pad_type]:
                prob_vs_dim_objs[hitmap_key][pad_type][hitmap_type] = {}

            frame = ROOT.TH2D(f"frame_{hitmap_key}_{pad_type}_{hitmap_type}", f"{angle}urad {angle_dir} {int(beta*100)}cm Station {station} {pad_title}{hitmap_title}", 100, 10, 10000, 100, 0.001, 1.1)
            frame.SetStats(False)
            frame.GetXaxis().SetTitle(x_title)
            frame.GetYaxis().SetTitle("Event Loss Probability")

            legend = ROOT.TLegend(0.65,0.1,0.9,0.45)

            prob_vs_dim_objs[hitmap_key][pad_type][hitmap_type]['frame'] = frame
            prob_vs_dim_objs[hitmap_key][pad_type][hitmap_type]['legend'] = legend

            idx += 1
            pad = canv.cd(idx)
            pad.SetLogx()
            pad.SetLogy()
            pad.SetTicks()

            frame.Draw()

            colorOffset = 0
            for timeStep in drawTimeSteps:
                probability = event_loss_probabilities[pad_type][hitmap_key][hitmap_type][f'prob_timeStep{timeStep}']
                length      = event_loss_probabilities[pad_type][hitmap_key][hitmap_type]['length']

                x, y = array( 'd' ), array( 'd' )
                for ibin in range(len(length)):
                    x.append(length[ibin])
                    y.append(probability[ibin])
                graph = ROOT.TGraph(len(length), x, y)

                graph.SetLineColor(ROOT.kAzure + colorOffset)
                graph.Draw("l same")
                legend.AddEntry(graph, "#tau #leq {} ns".format((timeStep+1)*25), "l")
                colorOffset += 3

                prob_vs_dim_objs[hitmap_key][pad_type][hitmap_type][f'graph_timeStep{timeStep}'] = graph

            legend.Draw()

    canv.Draw()
    prob_vs_dim_canvas[hitmap_key] = canv

### Reference Points for Pad Sizing

#### Pad Size Needed for 10% efficiency with 25 ns deadtime

In [ ]:
table = []
rows = []
columns = []

for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    do_continue = False
    for pad_type in occupancy_graphs:
        if hitmap_key not in occupancy_graphs[pad_type]:
            do_continue = True
            break
    if do_continue:
        continue

    these_columns = []
    this_row = []
    for pad_type in event_loss_probabilities:
        pad_title = "Square"
        if pad_type == "rectangular":
            pad_title = "Rect"

        for hitmap_type in event_loss_probabilities[pad_type][hitmap_key]:
            hitmap_title = ""
            if hitmap_type == 'background':
                hitmap_title = f" w/ Bkg"

            these_columns += [f"{pad_title}{hitmap_title}"]

            probability = event_loss_probabilities[pad_type][hitmap_key][hitmap_type][f'prob_timeStep0']
            length      = event_loss_probabilities[pad_type][hitmap_key][hitmap_type]['length']

            this_row += [numpy.interp(0.1, probability, length)]

    if columns == []:
        columns = these_columns
    if these_columns != columns:
        print("There is a problem with the ordering")

    table += [this_row]
    rows += [f"{angle_dir} {angle}urad {int(beta*100)}cm"]

df = pandas.DataFrame(table, columns = columns, index=rows)
print(df)

Units above in $\mu m$

#### Loss Probability for different pad sizes with 25 ns deadtime

In [ ]:
table2 = []
table3 = []
table4 = []
rows = []
columns = []
for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    do_continue = False
    for pad_type in occupancy_graphs:
        if hitmap_key not in occupancy_graphs[pad_type]:
            do_continue = True
            break
    if do_continue:
        continue

    these_columns = []
    this_row2 = []
    this_row3 = []
    this_row4 = []
    for pad_type in event_loss_probabilities:
        pad_title = "Square"
        if pad_type == "rectangular":
            pad_title = "Rect"

        for hitmap_type in event_loss_probabilities[pad_type][hitmap_key]:
            hitmap_title = ""
            if hitmap_type == 'background':
                hitmap_title = f" w/ Bkg"

            these_columns += [f"{pad_title}{hitmap_title}"]

            probability = event_loss_probabilities[pad_type][hitmap_key][hitmap_type][f'prob_timeStep0']
            length      = event_loss_probabilities[pad_type][hitmap_key][hitmap_type]['length']

            this_row2 += [numpy.interp(1300/2, length, probability)]
            this_row3 += [numpy.interp(1300/3, length, probability)]
            this_row4 += [numpy.interp(1300/4, length, probability)]

    if columns == []:
        columns = these_columns
    if these_columns != columns:
        print("There is a problem with the ordering")

    table2 += [this_row2]
    table3 += [this_row3]
    table4 += [this_row4]
    rows += [f"{angle_dir} {angle}urad {int(beta*100)}cm"]

df2 = pandas.DataFrame(table2, columns = columns, index=rows)
df3 = pandas.DataFrame(table3, columns = columns, index=rows)
df4 = pandas.DataFrame(table4, columns = columns, index=rows)

print(f"Pad dimension 1/2 of nominal ({1.3/2})")
print(df2)
print()

print(f"Pad dimension 1/3 of nominal ({1.3/3})")
print(df3)
print()

print(f"Pad dimension 1/4 of nominal ({1.3/4})")
print(df4)
print()

## Vertical Shifts needed over time

Assuming 1 year of HL-LHC is $300 fb^{-1}$ and LGAD limit of 2e15 neq/cm^2

In [ ]:
lgad_limit = 2e15
luminosity = 300

### Vertical Crossing Angle, 250 urad

#### Vertical crossing angle, 250 urad; 50 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "vertical-250urad-50cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Vertical crossing angle, 250 urad; 50 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "vertical-250urad-50cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Vertical crossing angle, 250 urad; 20 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "vertical-250urad-20cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Vertical crossing angle, 250 urad; 20 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "vertical-250urad-20cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

### Vertical Crossing Angle, 125 urad

#### Vertical crossing angle, 125 urad; 50 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "vertical-125urad-50cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Vertical crossing angle, 125 urad; 50 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "vertical-125urad-50cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Vertical crossing angle, 125 urad; 20 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "vertical-125urad-20cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Vertical crossing angle, 125 urad; 20 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "vertical-125urad-20cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

### Horizontal Crossing Angle, 250 urad

#### Horizontal crossing angle, 250 urad; 50 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "horizontal-250urad-50cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Horizontal crossing angle, 250 urad; 50 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "horizontal-250urad-50cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Horizontal crossing angle, 250 urad; 15 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "horizontal-250urad-15cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Horizontal crossing angle, 250 urad; 15 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "horizontal-250urad-15cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

### Horizontal Crossing Angle, 125 urad

#### Horizontal crossing angle, 125 urad; 50 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "horizontal-125urad-50cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Horizontal crossing angle, 125 urad; 50 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "horizontal-125urad-50cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Horizontal crossing angle, 125 urad; 15 cm $\beta^*$; no Background

In [ ]:
hitmap_id = "horizontal-125urad-15cm"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

#### Horizontal crossing angle, 125 urad; 15 cm $\beta^*$; with Background

In [ ]:
hitmap_id = "horizontal-125urad-15cm-background"
if hitmap_id in hitmaps:
    canv, persistance = hitmaps[hitmap_id].plotShifts(
        integratedLuminosity=luminosity,
        thresholdFlux=2*lgad_limit,
        plotPadCols=3,
        maxNumShifts=9,
    )
    canv.Draw()

### Needed Shifts

In [ ]:
needed_shifts = Helper20240606.getNeededShifts(station)
for key in needed_shifts:
    print(f"{key} - {needed_shifts[key][0]} shifts over {needed_shifts[key][1]} mm")

### Compute positions with shifts

In [ ]:
yOffsets = {}
for angle_dir in ["vertical", "horizontal"]:
    yPositions = []
    for angle in [125, 250]:
        position_key = f"{angle_dir}-{angle}urad"

        _, startPositionY = adjusted_positions[position_key]
        yPositions += [startPositionY]

        yCenters = Helper20240606.computeYCenters234(startPositionY, angle_dir, ySensorSize)
        yOffsets[position_key] = Helper20240606.computeShifts(yCenters, needed_shifts[position_key])

    positionY = max(yPositions)
    yCenters = Helper20240606.computeYCenters234(positionY, angle_dir, ySensorSize)
    yOffsets[angle_dir] = Helper20240606.computeShifts(yCenters, needed_shifts[angle_dir])

print(yOffsets)

### Sensor Positions on Hitmap with shifts

In [ ]:
shift_str = "{angle_dir}"
#shift_str = "{angle_dir}-{angle}urad"

In [ ]:
def plotShiftsOnFluxmap(angle_dir, angle, beta):
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    return Helper20240606.plotShiftsOnFluxmap(
        sensor = sensor,
        positions = yOffsets[shift_str.format(angle_dir = angle_dir, angle = angle, beta = beta)],
        hitmap = hitmaps[hitmap_key],
        histogram = hitmap_histograms[hitmap_key]["physics"],
        base_name = hitmap_key,
        title = f"{angle_dir} ${angle} \\mu\\text{{rad}}$ - $\\beta^* = {int(beta*100)} \\text{{cm}}$",
    )

#### Vertical crossing angle, 250 urad

##### Vertical crossing angle, 250 urad; 50 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("vertical", 250, 0.50)

canvas.Draw()

##### Vertical crossing angle, 250 urad; 20 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("vertical", 250, 0.20)

canvas.Draw()

#### Vertical crossing angle, 125 urad

##### Vertical crossing angle, 125 urad; 50 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("vertical", 125, 0.50)

canvas.Draw()

##### Vertical crossing angle, 125 urad; 20 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("vertical", 125, 0.20)

canvas.Draw()

#### Horizontal crossing angle, 250 urad

##### Horizontal crossing angle, 250 urad; 50 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("horizontal", 250, 0.50)

canvas.Draw()

##### Horizontal crossing angle, 250 urad; 15 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("horizontal", 250, 0.15)

canvas.Draw()

#### Horizontal crossing angle, 125 urad

##### Horizontal crossing angle, 125 urad; 50 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("horizontal", 125, 0.50)

canvas.Draw()

##### Horizontal crossing angle, 125 urad; 15 cm $\beta^*$

In [ ]:
canvas, persistance = plotShiftsOnFluxmap("horizontal", 125, 0.15)

canvas.Draw()

## Sensor Occupancy and Loss Probability

In [ ]:
coverage_shifts = 4
coverage_shift_step = 1.3/4
coverage_positions = {}

for angle_dir, angle, beta in angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    table_data = []
    rows = []
    cols = ["Dose", "Occ", "Loss p", "Pad", "Dose w/ Bckg", "Occ w/ Bckg", "Loss p w/ Bckg", "Pad w/ Bckg"]

    hitmap      = hitmaps[hitmap_key]
    hitmap_bckg = hitmaps[hitmap_key + "-background"]

    if hitmap_key not in coverage_positions:
        coverage_positions[hitmap_key] = []
        center_position_y = yOffsets[shift_str.format(angle_dir = angle_dir, angle = angle, beta = beta)][0][0]
        center_position_x = hitmap.detectorEdge * 1000 + xSensorSize/2
        center_position_y = adjusted_positions[f"{angle_dir}-{angle}urad"][1]
        for step in range(coverage_shifts):
            coverage_positions[hitmap_key] += [(center_position_x, center_position_y + step*coverage_shift_step)]

    for step in range(coverage_shifts):
        current_position = [coverage_positions[hitmap_key][step]]
        sensor.setShifts(current_position)

        # Calculations without Background
        sensor.calculateFlux(hitmap)
        dose      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies, pads = sensor.findMaxOccupancy()
        occupancy = occupancies[0]
        pad = pads[0]
        loss_prob = pps_hitmaps.calcEventLossProb(0, occupancy)

        # Calculations with Background
        sensor.calculateFlux(hitmap_bckg)
        dose_bckg      = sensor.maxDoseEOL(integratedLuminosity=1, usePadSpacing=False)
        occupancies_bckg, pads_bckg = sensor.findMaxOccupancy()
        occupancy_bckg = occupancies_bckg[0]
        pad_bckg = pads_bckg[0]
        loss_prob_bckg = pps_hitmaps.calcEventLossProb(0, occupancy_bckg)

        rows += [f"Offset {step}"]
        table_data += [[dose, occupancy, loss_prob, pad, dose_bckg, occupancy_bckg, loss_prob_bckg, pad_bckg]]

    df = pandas.DataFrame(table_data, columns = cols, index=rows)
    print(f"{angle_dir} {angle} urad - {int(beta*100)} cm:")
    print(df)
    print()

In [ ]:
def plotMaps(angle_dir, angle, beta):
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    hitmap = hitmaps[hitmap_key + "-background"]

    coverage_shifts = 4
    coverage_shift_step = 1.3/4

    canvas = []
    persistance = []

    positions = yOffsets[shift_str.format(angle_dir = angle_dir, angle = angle, beta = beta)]
    edge = hitmap.detectorEdge * 1000 # Convert to mm for drawing
    offsetX = edge + xSensorSize/2
    for sensor_idx in range(len(positions)):
        offsetY = positions[sensor_idx][0]

        coverage_positions = [
            (offsetX, offsetY + i*coverage_shift_step)
            for i in range(coverage_shifts)
            ]

        sensor.setShifts(coverage_positions)
        sensor.calculateFlux(hitmap)

        c, p = sensor.plotSensorQuantity("occupancy")
        c.SetName(c.GetName() + f"_sensor{sensor_idx}")
        canvas.append(c)
        persistance.append(p)

        c, p = sensor.plotSensorQuantity("loss_probability")
        c.SetName(c.GetName() + f"_sensor{sensor_idx}")
        canvas.append(c)
        persistance.append(p)

    return canvas, persistance

### Vertical crossing angle, $250 \, \mu rad$

#### Vertical crossing angle, $250 \, \mu rad$; $50 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("vertical", 250, 0.50)

for c in canvas:
    c.Draw()

#### Vertical crossing angle, $250 \, \mu rad$; $20 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("vertical", 250, 0.20)

for c in canvas:
    c.Draw()

### Vertical crossing angle, $125 \, \mu rad$

#### Vertical crossing angle, $125 \, \mu rad$; $50 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("vertical", 125, 0.50)

for c in canvas:
    c.Draw()

#### Vertical crossing angle, $125 \, \mu rad$; $20 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("vertical", 125, 0.20)

for c in canvas:
    c.Draw()

### Horizontal crossing angle, $250 \, \mu rad$

#### Horizontal crossing angle, $250 \, \mu rad$; $50 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("horizontal", 250, 0.50)

for c in canvas:
    c.Draw()

#### Horizontal crossing angle, $250 \, \mu rad$; $15 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("horizontal", 250, 0.15)

for c in canvas:
    c.Draw()

### Horizontal crossing angle, $125 \, \mu rad$

#### Horizontal crossing angle, $125 \, \mu rad$; $50 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("horizontal", 125, 0.50)

for c in canvas:
    c.Draw()

#### Horizontal crossing angle, $125 \, \mu rad$; $15 \, cm \; \beta^*$

In [ ]:
canvas, persistance = plotMaps("horizontal", 125, 0.15)

for c in canvas:
    c.Draw()

## Next